In [1]:
# Use scikit-learn to grid search the batch size and epochs
import numpy
import pandas as pd
from matplotlib import pyplot
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from keras.layers import LSTM
from keras.utils.vis_utils import plot_model

/home/qzhq7441/code/sckit-learn/hands-on-ml/venv/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# Function to create model, required for KerasClassifier
n_features = 1 
def create_model():
    model = Sequential()
    model.add(LSTM(4, activation='relu', input_shape=(n_steps, n_features)))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    model.summary()
    return model

In [3]:
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

In [4]:
#load dataset
series = pd.read_csv('Daily_TSNew_i-001683a6-rp-static-server-15197471722.csv', index_col=[0])
series = series.values


In [5]:
#Split the data into train and test 
train_size = int(len(series) * 0.67)
train, test = series[0:train_size,:], series[train_size:len(series),:]
train.shape, test.shape

((17360, 1), (8551, 1))

In [12]:
# Create your Supervised Learning Table
from numpy import array
import numpy
# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-1:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)

n_steps=1
#create trainseriesX and trainseriesy
trainseriesX, trainseriesy = split_sequence(train, n_steps)
#create testseriesX and testseriesy
testseriesX, testseriesy = split_sequence(test, n_steps)
trainseriesX.shape, trainseriesy.shape
print (trainseriesX.shape, testseriesX.shape)
# reshape of the train ad test in the 
trainseriesX.shape,

(17359, 1, 1) (8550, 1, 1)


((17359, 1, 1),)

In [7]:
# create model
model = KerasRegressor(build_fn=create_model, verbose=0)
# define the grid search parameters
batch_size = [10, 20, 40, 60, 80, 100]
epochs = [10, 50, 100]
param_grid = dict(batch_size=batch_size, epochs=epochs)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)

In [8]:
#grid.fit?

In [9]:
grid_result = grid.fit(trainseriesX, trainseriesy)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 4)                 96        
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 5         
Total params: 101
Trainable params: 101
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 4)                 96        
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 4)                 96        
____________

Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 4)                 96        
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 5         
Total params: 101
Trainable params: 101
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_8 (LSTM)                (None, 4)                 96        
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 5         
Total params: 101
Trainable params: 101
Non-trainable params: 0
________________________________________________________

Total params: 101
Trainable params: 101
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_8 (LSTM)                (None, 4)                 96        
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 5         
Total params: 101
Trainable params: 101
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 4)                 96        
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 5         
Total params: 101
Trainable params: 101
Non-trainable params: 0
________________

Total params: 101
Trainable params: 101
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_11 (LSTM)               (None, 4)                 96        
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 5         
Total params: 101
Trainable params: 101
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_7 (LSTM)                (None, 4)                 96        
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 5         
Total params: 101
Trainable params: 101
Non-trainable params: 0
________________

In [10]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: -2.891755 using {'batch_size': 40, 'epochs': 100}
-989.365935 (280.746856) with: {'batch_size': 10, 'epochs': 10}
-404.408374 (140.261383) with: {'batch_size': 10, 'epochs': 50}
-199.485544 (277.651041) with: {'batch_size': 10, 'epochs': 100}
-3.023976 (0.337076) with: {'batch_size': 20, 'epochs': 10}
-383.486940 (538.126473) with: {'batch_size': 20, 'epochs': 50}
-106.998765 (147.109875) with: {'batch_size': 20, 'epochs': 100}
-421.387120 (592.147203) with: {'batch_size': 40, 'epochs': 10}
-502.975206 (381.334327) with: {'batch_size': 40, 'epochs': 50}
-2.891755 (0.438308) with: {'batch_size': 40, 'epochs': 100}
-515.250026 (724.338581) with: {'batch_size': 60, 'epochs': 10}
-566.614931 (439.340273) with: {'batch_size': 60, 'epochs': 50}
-335.803417 (470.567390) with: {'batch_size': 60, 'epochs': 100}
-3.514957 (0.755282) with: {'batch_size': 80, 'epochs': 10}
-454.908919 (639.157986) with: {'batch_size': 80, 'epochs': 50}
-571.345486 (461.497823) with: {'batch_size': 80, 'epoch

### Best: -2.891755 using {'batch_size': 40, 'epochs': 100}